# Project 1: Benson.  MTA Outreach Analysis for WTWY

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from IPython.display import Image

In [2]:
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

## Read in and explore raw data

In [4]:
TURNSTILE_DIR = '../data/mta.info/turnstile/'
raw_df = pd.read_csv(TURNSTILE_DIR + 'turnstile_180113.txt',
                     skiprows=1,
                     names=['ControlArea', 'Unit', 'SCP', 'Station',
                            'LineName', 'Division', 'Date', 'Time',
                            'Description', 'Entries', 'Exits'],
                     dtype={'Entries': np.int, 'Exits': np.int},
                     parse_dates=[6],
                     thousands=',')

In [5]:
raw_df.shape

(201118, 11)

In [6]:
raw_df.dtypes

ControlArea            object
Unit                   object
SCP                    object
Station                object
LineName               object
Division               object
Date           datetime64[ns]
Time                   object
Description            object
Entries                 int64
Exits                   int64
dtype: object

In [7]:
raw_df.sample(5)

,ControlArea,Unit,SCP,Station,LineName,Division,Date,Time,Description,Entries,Exits
2101,A014,R081,02-06-00,49 ST,NQRW,BMT,2018-01-09,07:00:00,REGULAR,617256,2460168
125730,R101,R001,02-06-00,SOUTH FERRY,1RW,IRT,2018-01-10,08:40:32,REGULAR,929,0
119258,PTH16,R550,01-00-00,LACKAWANNA,1,PTH,2018-01-12,14:24:22,REGULAR,15973,59420
7033,A043,R462,00-03-03,CANAL ST,JNQRZ6W,BMT,2018-01-08,08:00:00,REGULAR,3136995,1521625
129556,R122,R290,02-05-01,HOUSTON ST,1,IRT,2018-01-09,00:00:00,REGULAR,50331648,33554542


In [8]:
len(raw_df['Time'].unique())

12161

* **Observation:** Some times are on hour-boundaries, but many are not.

In [9]:
raw_df['DateTime'] = [dt.datetime.strptime(d + ' ' + t, '%Y-%m-%d %H:%M:%S')
                      for d,t in zip(raw_df['Date'].astype(str), raw_df['Time'])]

In [10]:
hourly_mask = (raw_df['DateTime'].dt.minute == 0) & (raw_df['DateTime'].dt.second == 0)

In [11]:
hourly_mask.value_counts()

True     180780
False     20338
Name: DateTime, dtype: int64

* **Observation:** About 90% of the data is recorded on the hour.

In [12]:
sample1_mask = ((raw_df['Date'] == '2018-01-12')
        & (raw_df['Time'] == '04:00:00')
        & (raw_df['Unit'] == 'R173'))
raw_df[sample1_mask]

,ControlArea,Unit,SCP,Station,LineName,Division,Date,Time,Description,Entries,Exits,DateTime
41494,N001,R173,01-05-00,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,301,0,2018-01-12 04:00:00
41536,N001,R173,01-05-01,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,67108872,16777412,2018-01-12 04:00:00
41578,N001,R173,01-06-00,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,4143559,760241,2018-01-12 04:00:00
41620,N001,R173,01-06-01,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,201806894,302650,2018-01-12 04:00:00
41662,N001,R173,01-06-02,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,742566,553216,2018-01-12 04:00:00
41704,N001,R173,01-06-03,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,332417,162116,2018-01-12 04:00:00
41746,N002A,R173,00-00-00,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,5655675,3308405,2018-01-12 04:00:00
41788,N002A,R173,00-00-01,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,9247893,5364605,2018-01-12 04:00:00
41830,N002A,R173,00-00-02,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,6382433,3683969,2018-01-12 04:00:00
41872,N002A,R173,00-05-00,INWOOD-207 ST,A,IND,2018-01-12,04:00:00,REGULAR,3414,0,2018-01-12 04:00:00


In [13]:
sample2_mask = ((raw_df['Date'] == '2018-01-12')
        & (raw_df['Time'] == '08:00:00')
        & (raw_df['Unit'] == 'R173'))
raw_df[sample2_mask]

,ControlArea,Unit,SCP,Station,LineName,Division,Date,Time,Description,Entries,Exits,DateTime
41495,N001,R173,01-05-00,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,301,0,2018-01-12 08:00:00
41537,N001,R173,01-05-01,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,67108872,16777412,2018-01-12 08:00:00
41579,N001,R173,01-06-00,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,4144041,760256,2018-01-12 08:00:00
41621,N001,R173,01-06-01,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,201807042,302674,2018-01-12 08:00:00
41663,N001,R173,01-06-02,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,742751,553278,2018-01-12 08:00:00
41705,N001,R173,01-06-03,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,332668,162141,2018-01-12 08:00:00
41747,N002A,R173,00-00-00,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,5656200,3308462,2018-01-12 08:00:00
41789,N002A,R173,00-00-01,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,9248297,5364632,2018-01-12 08:00:00
41831,N002A,R173,00-00-02,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,6382835,3684013,2018-01-12 08:00:00
41873,N002A,R173,00-05-00,INWOOD-207 ST,A,IND,2018-01-12,08:00:00,REGULAR,3414,0,2018-01-12 08:00:00


* **Observation:** Some turnstiles don't have increasing Entries/Exists values, due to disuse or malfunction.

## Find traffic counts per time period

In [14]:
# raw_df.groupby(['ControlArea', 'Unit', 'SCP', 'Station'])

delta_df = raw_df.copy()
delta_df = delta_df.drop(['LineName', 'Division', 'Description', 'Time'], axis=1)
delta_df.set_index(['ControlArea', 'Unit', 'SCP', 'Station'], inplace=True)
delta_df.sort_values(by=['DateTime'])

delta_df['dt_delta'] = delta_df['DateTime'].diff()
delta_df['entries_delta'] = delta_df['Entries'].diff()
delta_df['exits_delta'] = delta_df['Exits'].diff()
# delta_df = delta_df.drop(['Exits', 'Entries'], axis=1)  # Will use below

delta_df.head()

Date  Entries    Exits  \
ControlArea Unit SCP      Station                                
A002        R051 02-00-00 59 ST   2018-01-06  6470216  2190123   
                          59 ST   2018-01-06  6470223  2190140   
                          59 ST   2018-01-06  6470256  2190229   
                          59 ST   2018-01-06  6470379  2190299   
                          59 ST   2018-01-06  6470665  2190366   

                                             DateTime dt_delta  entries_delta  \
ControlArea Unit SCP      Station                                               
A002        R051 02-00-00 59 ST   2018-01-06 03:00:00      NaT            NaN   
                          59 ST   2018-01-06 07:00:00 04:00:00            7.0   
                          59 ST   2018-01-06 11:00:00 04:00:00           33.0   
                          59 ST   2018-01-06 15:00:00 04:00:00          123.0   
                          59 ST   2018-01-06 19:00:00 04:00:00          286.0   

                                   exits_delta  
ControlArea Unit SCP      Station               
A002        R051 02-00-00 59 ST            NaN  
                          59 ST           17.0  
                          59 ST           89.0  
                          59 ST           70.0  
                          59 ST           67.0

In [15]:
len(delta_df['dt_delta'].unique())

1328

* **Observation:** Just as the times are irregular, so are the time intervals per turnstile.

In [16]:
print('Rows: {}'.format(delta_df.shape[0]))
print('Rows w/ entries_delta < 0: {}'.format(
    len([True for delta in delta_df['entries_delta'] if delta < 0])))
print('Rows w/ exits_delta < 0: {}'.format(
    len([True for delta in delta_df['exits_delta'] if delta < 0])))

Rows: 201118
Rows w/ entries_delta < 0: 3944
Rows w/ exits_delta < 0: 3650


In [17]:
print('Rows w/ entries_delta > 100,000,000: {}'.format(
    len([True for delta in delta_df['entries_delta'] if delta > 100_000_000])))
print('Rows w/ exits_delta > 100,000,000: {}'.format(
    len([True for delta in delta_df['exits_delta'] if delta > 100_000_000])))

Rows w/ entries_delta > 100,000,000: 217
Rows w/ exits_delta > 100,000,000: 142


* **Observation:** There is definitely some bad data here.  (Probably the result of diff'ing between groups.)

In [18]:
print('entries_delta: 99th percentile: {}'.format(delta_df['entries_delta'].quantile(q=0.99)))
print('entries_delta: 98th percentile: {}'.format(delta_df['entries_delta'].quantile(q=0.98)))
print('entries_delta: 97th percentile: {}'.format(delta_df['entries_delta'].quantile(q=0.97)))
print('entries_delta: 96th percentile: {}'.format(delta_df['entries_delta'].quantile(q=0.96)))
print('entries_delta: 95th percentile: {}'.format(delta_df['entries_delta'].quantile(q=0.95)))

entries_delta: 99th percentile: 208354.47999999704
entries_delta: 98th percentile: 1259.0
entries_delta: 97th percentile: 996.5199999999895
entries_delta: 96th percentile: 838.359999999986
entries_delta: 95th percentile: 738.0


In [19]:
print('exits_delta: 99th percentile: {}'.format(delta_df['exits_delta'].quantile(q=0.99)))
print('exits_delta: 98th percentile: {}'.format(delta_df['exits_delta'].quantile(q=0.98)))
print('exits_delta: 97th percentile: {}'.format(delta_df['exits_delta'].quantile(q=0.97)))
print('exits_delta: 96th percentile: {}'.format(delta_df['exits_delta'].quantile(q=0.96)))
print('exits_delta: 95th percentile: {}'.format(delta_df['exits_delta'].quantile(q=0.95)))

exits_delta: 99th percentile: 78240.75999999986
exits_delta: 98th percentile: 1138.679999999993
exits_delta: 97th percentile: 844.0
exits_delta: 96th percentile: 691.0
exits_delta: 95th percentile: 593.0


## Remove rows with questionable traffic counts

In [20]:
filter_mask = ((delta_df['entries_delta'] >= 0)
               & (delta_df['exits_delta'] >= 0)
               & (delta_df['entries_delta'] <= delta_df['entries_delta'].quantile(q=0.98))
               & (delta_df['exits_delta'] <= delta_df['exits_delta'].quantile(q=0.98))
              )

In [21]:
filter_mask.value_counts()

True     190715
False     10403
dtype: int64

In [22]:
filtered_df = delta_df.copy()
filtered_df = filtered_df[filter_mask]
filtered_df.drop(['Entries', 'Exits'], axis=1, inplace=True)

## Hourly chart

In [48]:
hourly_filtered_mask = ((delta_df['DateTime'].dt.minute == 0)
                     & (delta_df['DateTime'].dt.second == 0)
                    )

In [49]:
hourly_filtered_grp = filtered_df[hourly_filtered_mask].groupby(['Station', 'DateTime'])
hourly_entries_df = pd.DataFrame(hourly_filtered_grp['entries_delta'].aggregate('sum').astype(int))
hourly_exits_df = pd.DataFrame(hourly_filtered_grp['exits_delta'].aggregate('sum').astype(int))

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Exception: cannot handle a non-unique multi-index!

In [46]:
hourly_entries_df.hist()

NameError: name 'hourly_entries_df' is not defined

In [47]:
hourly_exits_df.hist()

NameError: name 'hourly_exits_df' is not defined

## Q: Can we identify which stations are in a business district?
## A: Not with certainty.

In [23]:
morning_mask = (delta_df['DateTime'].dt.hour >= 8) & (delta_df['DateTime'].dt.hour <= 10)

In [24]:
morning_mask.value_counts()

False    183048
True      18070
Name: DateTime, dtype: int64

In [25]:
evening_mask = (delta_df['DateTime'].dt.hour >= 16) & (delta_df['DateTime'].dt.hour <= 18)

In [26]:
evening_mask.value_counts()

False    186237
True      14881
Name: DateTime, dtype: int64

* **Note:** We're filtering out ~5% of the rows.

In [27]:
morning_station_df = delta_df[morning_mask].groupby(['Station']).sum()

In [28]:
evening_station_df = delta_df[evening_mask].groupby(['Station']).sum()

In [29]:
morning_business_stations_mask = morning_station_df['entries_delta'] < morning_station_df['exits_delta']
morning_business_stations = set(morning_station_df[morning_business_stations_mask].index)

In [30]:
evening_business_stations_mask = evening_station_df['entries_delta'] > evening_station_df['exits_delta']
evening_business_stations = set(evening_station_df[evening_business_stations_mask].index)

In [31]:
len(morning_business_stations)

69

In [32]:
len(evening_business_stations)

165

In [33]:
len(morning_business_stations - evening_business_stations)

29

In [34]:
len(evening_business_stations - morning_business_stations)

125

## Traffic per station

In [35]:
station_df = filtered_df.copy()
station_df['traffic'] = station_df['entries_delta'] + station_df['exits_delta']
station_df.drop(['dt_delta', 'entries_delta', 'exits_delta'], axis=1, inplace=True)
station_df.index.ControlArea = 'ControlArea'
station_df.index.Unit = 'Unit'
station_df.index.SCP = 'SCP'
station_df.reset_index(inplace=True)
station_df.drop(['ControlArea', 'Unit', 'SCP', 'Date', 'DateTime'], axis=1, inplace=True)

In [36]:
station_grouped = station_df.groupby('Station')

In [37]:
station_df = pd.DataFrame(station_grouped['traffic'].aggregate('sum').astype(int))

In [40]:
station_df.sort_values(by=['traffic'])

,traffic
Station,
BROAD CHANNEL,2153
BEACH 105 ST,3230
TOMPKINSVILLE,4423
BEACH 98 ST,5451
AVENUE I,5835
SENECA AVE,6427
ROCKAWAY PARK B,8187
BEACH 44 ST,8617
BEACH 90 ST,8783
